In [2]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
import os
from VotingCombination import VotingCombination
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
horizon = 12
window = 12
arima_results = './results/arima/rolling'
transformations = ["normal", "deseasonal", "log", "deseasonal-log"]

In [4]:
def get_predictions_csv(path, formats, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    params = {}
    for format in formats:
        filtered_df = df[df['DATA'] == format]
        
        columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
        
        # values_list = columns_p1_to_p12.values.tolist()
        values_list = columns_p1_to_p12.values.flatten().tolist()        
        results[format] = pd.Series(values_list, index=start_index)
        params[format] = filtered_df['PARAMS'].iloc[0]
            
    return results, params

In [7]:
results_mean = "./results/arima_mean/rolling"
results_median = "./results/arima_median/rolling"

colunas = ['DATA', 'MCPM', 'UF', 'PRODUCT', 'MODEL', 'PARAMS', 'WINDOW', 'HORIZON', 'RMSE', 'MAPE', 'POCID', 'PBE',
           'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12',
           ]
for subdir, _, files in os.walk(arima_results):
    for file in files:
        if file.endswith('.csv'):
            uf = file.split("_")[1][:2]
            derivado = subdir.split("/")[-1]
            file_path = os.path.join(subdir, file)
            series = read_series(f"../datasets/venda/mensal/uf/{derivado}/mensal_{uf.lower()}_{derivado}.csv")
            _, test = train_test_stats(series, horizon)
            

            predictions, params = get_predictions_csv(file_path, transformations, start_index=test.index)
            
            voting = VotingCombination(predictions, combination='mean')
            preds_mean = voting.predict()

            rmse_result = rmse(test, preds_mean)
            mape_result = mape(test, preds_mean)
            pocid_result = pocid(test, preds_mean)
            pbe_result = pbe(test, preds_mean)
            mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

            path_derivado = results_mean + f"/{derivado}"
            csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
            os.makedirs(path_derivado, exist_ok=True)

            df_result = pd.DataFrame({'DATA': "_".join(predictions.keys()), 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': 'ARIMA_MEAN', 'PARAMS': str(params), 'WINDOW': window, 'HORIZON': horizon,  
                                            'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result,
                                            'P1': preds_mean[0], 'P2': preds_mean[1], 'P3': preds_mean[2], 'P4': preds_mean[3], 'P5': preds_mean[4],
                                            'P6': preds_mean[5], 'P7': preds_mean[6], 'P8': preds_mean[7], 'P9': preds_mean[8], 'P10': preds_mean[9],
                                            'P11': preds_mean[10], 'P12': preds_mean[11]
                                            }, index=[0])
            df_result = df_result[colunas]

            df_result.to_csv(csv_file, index=False)


            #MEDIAN
            voting = VotingCombination(predictions, combination='median')
            preds_mean = voting.predict()

            rmse_result = rmse(test, preds_mean)
            mape_result = mape(test, preds_mean)
            pocid_result = pocid(test, preds_mean)
            pbe_result = pbe(test, preds_mean)
            mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

            path_derivado = results_median + f"/{derivado}"
            csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
            os.makedirs(path_derivado, exist_ok=True)
            df_result = pd.DataFrame({'DATA': "_".join(predictions.keys()), 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': 'ARIMA_MEAN', 'PARAMS': str(params), 'WINDOW': window, 'HORIZON': horizon,  
                                            'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result,
                                            'P1': preds_mean[0], 'P2': preds_mean[1], 'P3': preds_mean[2], 'P4': preds_mean[3], 'P5': preds_mean[4],
                                            'P6': preds_mean[5], 'P7': preds_mean[6], 'P8': preds_mean[7], 'P9': preds_mean[8], 'P10': preds_mean[9],
                                            'P11': preds_mean[10], 'P12': preds_mean[11]
                                            }, index=[0])
            df_result = df_result[colunas]

            df_result.to_csv(csv_file, index=False)


{'normal': "{'p': 10, 'd': 1, 'q': 2}", 'deseasonal': "{'p': 9, 'd': 1, 'q': 12}", 'log': "{'p': 10, 'd': 1, 'q': 2}", 'deseasonal-log': "{'p': 5, 'd': 1, 'q': 6}"}
{'normal': "{'p': 8, 'd': 1, 'q': 10}", 'deseasonal': "{'p': 6, 'd': 1, 'q': 14}", 'log': "{'p': 11, 'd': 1, 'q': 13}", 'deseasonal-log': "{'p': 13, 'd': 1, 'q': 14}"}
{'normal': "{'p': 13, 'd': 1, 'q': 14}", 'deseasonal': "{'p': 5, 'd': 1, 'q': 5}", 'log': "{'p': 8, 'd': 1, 'q': 2}", 'deseasonal-log': "{'p': 2, 'd': 1, 'q': 3}"}
{'normal': "{'p': 8, 'd': 1, 'q': 14}", 'deseasonal': "{'p': 11, 'd': 1, 'q': 2}", 'log': "{'p': 7, 'd': 1, 'q': 11}", 'deseasonal-log': "{'p': 7, 'd': 1, 'q': 9}"}
{'normal': "{'p': 14, 'd': 1, 'q': 14}", 'deseasonal': "{'p': 2, 'd': 1, 'q': 2}", 'log': "{'p': 10, 'd': 1, 'q': 14}", 'deseasonal-log': "{'p': 2, 'd': 1, 'q': 2}"}
{'normal': "{'p': 9, 'd': 1, 'q': 2}", 'deseasonal': "{'p': 5, 'd': 1, 'q': 6}", 'log': "{'p': 10, 'd': 1, 'q': 2}", 'deseasonal-log': "{'p': 9, 'd': 1, 'q': 3}"}
{'normal'